In [ ]:
%matplotlib widget 
# when facing trouble with `%matplotlib backend`, also try: inline, notebook, ipympl
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from qm1.grid import *
from qm1.wavefunction import *
from qm1.operators import *
from qm1.qmsystem import *
from qm1.eigensystem import *
import qm1
# remove automatic toolbars
mpl.rcParams['toolbar'] = 'None'


## grid
Set up the [grid](https://en.wikipedia.org/wiki/Regular_grid) on which the values of the wave functions are stored. Currently only equidistant grids are implemented.

The grid points for the indices $i=0,...,n-1$ are generated by the rule
$$
x_i = \left(i+ \frac{1}{2}\right)\Delta x + x_\text{min}
$$
with $\Delta x = \frac{x_\text{max}-x_\text{min}}{n}$. 

This yields the set of points
$$
x_\text{min} + \frac{\Delta x}{2}, x_\text{min} + \frac{3\Delta x}{2}, ...,  x_\text{max} - \frac{\Delta x}{2}
$$

### boundary conditions
The two standard [boundary conditions](https://en.wikipedia.org/wiki/Boundary_value_problem) `periodic` and `vanishing` are implemented. There is also `open`, which does not impose any condition on the boundaries.

In [ ]:
# [x_min, x_max] is interval the wave function is defined on

# simulation grid
x_min, x_max = -20., 20.             
# number of grid points
num = 500                           
# set boundary condition
# bc = 'open'
# bc = 'periodic'
bc = 'vanishing'
grid = UniformGrid(bc, x_min, x_max, num)

## wave functions
The wave function is represeted by a vector containing the values of the wave function on the grid.
$$
\Psi = (\Psi_0, ..., \Psi_{n-1}) = (\Psi(x_i))_{i=0,...,n-1}
$$
The `Wavefunction` class supports `+-` with itself, `*` with scalars and a scalar product operation with itself, as well as other operations, e.g. `conjugated`, `normalized`, etc.

In conjunction with operators `OperatorConst` or `OperatorTD` there are more possibilities discussed in the following.

`Wavefunction`s support visualization by the `show` method.

In [ ]:
%load_ext autoreload
%autoreload 2

# use a built-in wave function
gaussian = GaussianWavePackage(grid, mu=1., sigma=2.5, k=-1.)
gaussian.show(absphase=True)
gaussian.conjugated()
gaussian.show(absphase=True)

# custom wave function
wf = Wavefunction(grid)
wf.from_func(lambda x: (x**2-1.)*np.exp(-x**2/2.))
wf.show()

### potential
Set the potential representing the desired quantum system. There is a series of built-in systems:

`ConstPot`, `StepPot`, `BarrierPot`, `HarmonicPot`, `DeltaPot`, `DoubleDeltaPot`, `InterpolatePot`

Custom potentials can be set via a callable of signature `Callable[[float], float]`.

In [ ]:
# or set up a custom potential
def your_custom_potential(x:float)->float:
  your_parameter_1 = np.pi / 15.
  your_parameter_2 = np.sin(18)
  result = np.sin(x/(x_max-x_min)*np.pi)*your_parameter_1 + your_parameter_2
  return result
potential = your_custom_potential

# choose from different predefined potentials, e.g.
potential = DoubleDeltaPot(grid)

### mass
The mass is nothing one can manipulate usually. In Hartree automic units the electron mass is 1. In changing the mass 
some aspects of the time evolution appear more distinct. The mass strongly affects the diffusion, discriminating wave-like and particle like effects. 

In [ ]:
mass = 1.

## Quantum system
The class `QMSystem` handles most of the required code to represent a specific quantum system. It describes the particle, grid, stationary and time-depenedent potential.

In [ ]:
qsys = QMSystem(grid=grid, stat_pot=potential, mass=mass)

## operators
Operators are key in qm calculations. With a wave function as a vector the operators are cast in matrix form (see [Matrix mechanics](https://en.wikipedia.org/wiki/Matrix_mechanics)). `scipy` supports efficient classes for sparse matrices and is used to speed up the operator class `OperatorConst`. There are several built-in operators:

`IdentityOp`, `ZeroOp`, `PositionOp`, `GradientOp`, `MomentumOp`, `LaplaceOp`, `StatPotentialOp`,`KineticOp`


In [ ]:
op_identity = IdentityOp(qsys.grid)
op_position = PositionOp(qsys.grid)
op_momentum = MomentumOp(qsys.grid)
op_hamilton = HamiltonOp(qsys)
op_potential= StatPotentialOp(qsys)

#### custom operators 
Custom operators can also be made as shown in the following example.
Any operator can be constructed via `from_matrix` if its matrix representation `mat` is known.

In [ ]:
# instantiate
op_random = OperatorConst(grid=qsys.grid)
# fill `mat` with randoms
mat = np.random.rand(qsys.grid.num, qsys.grid.num)
# use `from_matrix`
op_random.from_matrix(mat)
# convert the operator to an efficient sparse respresentation
op_random.finalize()
print(op_random)
op_random.show()

### operator algebra
Operator algebra has to be understood in the framework of the matrix representation of finite grids.

For example the simple calculation of the commutator of position and momentum operators needs further interpretation with finite grids.

#### example: commutator of $x$ and $p$
The momentum operator is represented as 
$$(\hat{p})_{ij} = -\frac{i\hbar}{2\Delta x} (\delta_{i,j+1}-\delta_{i,j-1})$$
and the position operator as 
$$(\hat{x})_{ij} = x_i\delta_{i,j}$$
In matrix form the products of momentum and position operator read
$$
(\hat{xp})_{ij} = -\frac{i\hbar}{2\Delta x} \sum_k x_i\delta_{i,k} (\delta_{k,j+1}-\delta_{k,j-1}) = -\frac{i\hbar}{2\Delta x} x_i(\delta_{i,j+1}-\delta_{i,j-1}),
$$
and
$$
(\hat{px})_{ij} = -\frac{i\hbar}{2\Delta x} \sum_k (\delta_{i,k+1}-\delta_{i,k-1})x_k\delta_{k,j} = -\frac{i\hbar}{2\Delta x} (\delta_{i,j+1}-\delta_{i,j-1})x_j.
$$
Together the commutator $[\hat{x},\hat{p}]$ in matrix form reads
$$
(\hat{xp})_{ij}- (\hat{px})_{ij} = -\frac{i\hbar}{2\Delta x} \left( x_i(\delta_{i,j+1}-\delta_{i,j-1})-(\delta_{i,j+1}-\delta_{i,j-1})x_j\right)= -\frac{i\hbar}{2\Delta x}   (\delta_{i,j+1}-\delta_{i,j-1}) (x_i-x_j)  = \\
=-\frac{i\hbar}{2\Delta x} (-\Delta x\delta_{i,j+1}-\Delta x\delta_{i,j-1})  =  \frac{i\hbar}{2}(\delta_{i,j+1}+\delta_{i,j-1}) 
$$
Where the distance $x_i-x_j$ has been rewritten as $\pm\Delta x$ respecting the Kronecker-Deltas. This is not the expected result ($-i\hbar \delta_{ij}$) due to the representation of the operators on a finite gridspacing. However, as $\Delta x \to 0$ the action of the operator $[x,p]_{ij}$ resembles more and more that of $-i\hbar \delta_{ij}$ (with the general assumption of a continuous wave function).

Note: For the sake of simplicity the operator of the first derivative treated here, does not include proper finite difference coefficients at the grid boundary.

In [ ]:
op_commutator = op_position*op_momentum - op_momentum*op_position
print(op_position)
print(op_momentum)
print(op_commutator)
op_commutator.show()


## Operators acting on wave functions
### observables 
For any wave function expectation values and variances can be calculated via the methods `wf.expectation_value(op)` and `wf.variance(op)`

In [ ]:
print('exp. value of position operator:', wf.expectation_value(operator=op_position))
print('variance   of position operator:', wf.variance(operator=op_position))

print('exp. value of hamilton operator:', wf.expectation_value(operator=op_hamilton))
print('variance   of hamilton operator:', wf.variance(operator=op_hamilton))


## Eigen systems
For any hermitian operator there exists a real eigen system. From the eigen system the method `Eigensystem` calculates the `num` lowest eigen systes (by eigen value). 

Note that due to the finite matrix representation, numerical evaluation of eigen vectors gets difficult to analytically impossible when the number of requested eigen vectors exceeds the rank of the operator's matrix.

In [ ]:
from qm1.eigensystem import Eigensystem
import pandas

# number of states to calculate
num_states = 30

# get eigensystem 
eigsys = Eigensystem(num=num_states, operator=op_hamilton)
eigsys.show(file="tutorial_hamiltonian_eigensystem.png")
obs = eigsys.get_observables([op_identity, op_position, op_momentum, op_hamilton])
df = pandas.DataFrame(np.real(obs[:,:,0]), columns=['op_identity','op_position', 'op_momentum', 'op_hamilton'],  index=['state '+str(_is) for _is in range(eigsys.num)])
df

In [ ]:
from qm1.measurement import Measure
m = Measure(op=op_hamilton, num_states=num_states)
m(wf, num_obs=1000)
m.show()
